In [ ]:
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(filename)s:%(lineno)d - %(message)s",
)

from app.instructions import global_tools_hub
global_tools_hub.load_tools("tools")

In [ ]:
from notebooks.plan_mcts_optimizer import MCTSPlanOptimizer

# task_id = pending_tasks[0]['id']
task_id = "459f65c5-6671-4eae-bc41-16ff4e5b8aa4"

optimizer = MCTSPlanOptimizer(task_id=task_id, max_iterations=3, time_limit_seconds=900)

In [ ]:
print("optimizing task", task_id)

def print_node(node):
    print("*" * 100)
    print(node.state.seq_no, node.state.commit_hash, node.visits, node.value)
    for suggestion in node.optimization_suggestions:
        print(suggestion["branch_name"])
        print(suggestion["suggestion"])
    for child in node.children:
        print_node(child)

print_node(optimizer.root)

In [ ]:
optimizer.optimize()

In [ ]:
answers = optimizer.sort_final_answers()
answers

In [ ]:
from notebooks.tasks import save_best_plan_from_url

save_best_plan_from_url(task_id=task_id, commit_hash=answers[0]['commit_hash'])
# save_best_plan_from_url(task_id="c255c42b-c85d-450f-aa2b-b101f37adef8", commit_hash="53c56f51425643e296ac04bdebe751d1")

In [ ]:
from notebooks.tasks import record_evaluation,record_human_evaluation

record_evaluation(task_id, "APPROVED", f"The plan is optimized by the LLM, and choose the best answer among {len(answers)} answers.")
record_human_evaluation(task_id, "APPROVED", f"The plan is optimized by the LLM, and choose the best answer among {len(answers)} answers.")

In [ ]:
from notebooks.tasks import ts
from app.config.database import SessionLocal

tt_id = '459f65c5-6671-4eae-bc41-16ff4e5b8aa4'
keep_branches = ['main']
with SessionLocal() as session:
    tt = ts.get_task(session, tt_id)
    branches = tt.get_branches()
    for branch in branches:
        if branch['name'] not in keep_branches:
            print(f"deleting branch {branch['name']}")
            tt.delete_branch(branch['name'])
        else:
            print(f"keeping branch {branch['name']}")